## Siamese NN

In [1]:
import os
from tensorflow.keras.models import load_model
import tensorflow.keras.backend as K
from SNN.siamese_network import *
from SNN.siamese_network_parse import PrepareDataSNN
from sklearn.model_selection import train_test_split

2025-01-12 16:54:52.211695: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-12 16:54:52.919828: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Preparar la data

In [2]:
data = PrepareDataSNN()

# Ruta para guardar los datos
data_dir = "./data_snn"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

data_file = os.path.join(data_dir, "dataset_split.npz")

if os.path.exists(data_file):
    print("Cargando conjuntos de datos...")
    data = np.load(data_file)
    X1_train, X1_val, X1_test = data["X1_train"], data["X1_val"], data["X1_test"]
    X2_train, X2_val, X2_test = data["X2_train"], data["X2_val"], data["X2_test"]
    y_train, y_val, y_test = data["y_train"], data["y_val"], data["y_test"]
else:
    print("Procesando y dividiendo los datos...")
    data_a, data_b, labels, names = PrepareDataSNN().process()

    X1_temp, X1_test, X2_temp, X2_test, y_temp, y_test = train_test_split(
        data_a, data_b, labels, test_size=0.2, random_state=42
    )

    X1_train, X1_val, X2_train, X2_val, y_train, y_val = train_test_split(
        X1_temp, X2_temp, y_temp, test_size=0.25, random_state=42
    )

    np.savez(data_file, 
                X1_train=X1_train, X2_train=X2_train, y_train=y_train,
                X1_val=X1_val, X2_val=X2_val, y_val=y_val,
                X1_test=X1_test, X2_test=X2_test, y_test=y_test)

# Verificar las formas de los arrays
print("Formas de los conjuntos de entrenamiento:")
print(f"X1_train: {X1_train.shape}")
print(f"X2_train: {X2_train.shape}")
print(f"y_train: {y_train.shape}")

print("\nFormas de los conjuntos de validación:")
print(f"X1_val: {X1_val.shape}")
print(f"X2_val: {X2_val.shape}")
print(f"y_val: {y_val.shape}")


Cargando conjuntos de datos...
Formas de los conjuntos de entrenamiento:
X1_train: (586, 65)
X2_train: (586, 65)
y_train: (586,)

Formas de los conjuntos de validación:
X1_val: (196, 65)
X2_val: (196, 65)
y_val: (196,)


## Inicializar la red siamesa, entrenar y guardar el modelo

In [3]:
# Inicializar y entrenar la red
input_shape = (65,)
siamese_net = SiameseNeuralNetwork(input_shape)

print("Entrenando el modelo...")
history = siamese_net.train(X1_train, X2_train, y_train, validation_data=(X1_val, X2_val, y_val))

# Guardar el modelo
siamese_net._save_model("model_1")


Entrenando el modelo...
Epoch 1/20


/home/chony/.local/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.4789 - loss: 1.4550 - val_accuracy: 0.5051 - val_loss: 1.3152
Epoch 2/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5171 - loss: 1.3224 - val_accuracy: 0.5357 - val_loss: 1.2777
Epoch 3/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5598 - loss: 1.2842 - val_accuracy: 0.5306 - val_loss: 1.2300
Epoch 4/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5035 - loss: 1.2786 - val_accuracy: 0.5255 - val_loss: 1.2046
Epoch 5/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5301 - loss: 1.2234 - val_accuracy: 0.5306 - val_loss: 1.1900
Epoch 6/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5251 - loss: 1.2044 - val_accuracy: 0.5306 - val_loss: 1.1707
Epoch 7/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5500 - loss: 1.1776 - val_accuracy: 0.5102 - val_loss: 1.1358
Epoch 8/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5209 - loss: 1.1610 - val_accuracy: 0.5153 - val_loss: 1.1280
Ep

## Cargar el modelo guardado

In [4]:
model = load_model('./models/model_1.keras', custom_objects={'L1Distance': L1Distance})

model.summary()

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 65)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, 65)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (None, 128)       │    201,600 │ input_layer_1[0]… │
│ (Sequential)        │                   │            │ input_layer_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ l1_distance         │ (None, 128)       │          0 │ sequential[0][0], │
│ (L1Distance)        │                   │            │ sequential[1][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 1)         │        129 │ l1_distance[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 601,605 (2.29 MB)

 Trainable params: 199,937 (781.00 KB)

 Non-trainable params: 1,792 (7.00 KB)

 Optimizer params: 399,876 (1.53 MB)

## Preparar la nueva data de validacion (Proyectos de C#)

In [5]:

# Ruta para guardar los datos
data_dir = "./data_val"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

data_file = os.path.join(data_dir, "dataset_split.npz")

if os.path.exists(data_file):
    # Cargar los datos si ya están guardados
    print("Cargando conjuntos de validación desde archivo...")
    data = np.load(data_file)
    X1_new, X2_new, y_new, names = data["X1_new"], data["X2_new"], data["y_new"], data["names"]
else:
    print("Procesando los datos de validación y guardándolos...")
    data_val = PrepareDataSNN("data/features_vect_val/")
    X1_new, X2_new, y_new, names = data_val.process()
    
    # Guardar los datos procesados
    np.savez(data_file, X1_new=X1_new, X2_new=X2_new, y_new=y_new, names=names)


# Verificar las formas de los arrays
print("Formas de los conjuntos de validación:")
print(f"X1_new: {X1_new.shape}")
print(f"X2_new: {X2_new.shape}")
print(f"y_new: {y_new.shape}")


Procesando los datos de validación y guardándolos...
Formas de los conjuntos de validación:
X1_new: (140, 65)
X2_new: (140, 65)
y_new: (140,)


## Evaluar el modelo con los datos nuevos

In [23]:
loss, accuracy = model.evaluate([X1_test, X2_test], y_test, verbose=1)
print(f"Pérdida en los nuevos datos: {loss:.4f}")
print(f"Accuracy en los nuevos datos: {accuracy:.4f}")
        
threshold = 0.10  # Ajusta este valor según los resultados que obtengas
predictions = (model.predict([X1_test, X2_test]) > threshold).astype(int).flatten()

# Calcular la matriz de confusión
tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()

# Calcular métricas adicionales
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)

# Imprimir las métricas adicionales
print(f"Verdaderos Positivos (TP): {tp}")
print(f"Falsos Positivos (FP): {fp}")
print(f"Verdaderos Negativos (TN): {tn}")
print(f"Falsos Negativos (FN): {fn}")
print(f"Precisión (Precision): {precision:.4f}")
print(f"Exhaustividad (Recall): {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

# Registrar pares que resultaron en FN y VP
falsos_negativos = []
falsos_positivos = []
verdaderos_positivos = []

if names is not None:
    for pair, true, pred in zip(names, y_test, predictions):
        if true == 1 and pred == 0:
            falsos_negativos.append(pair)
        elif true == 0 and pred == 1:
            falsos_positivos.append(pair)
        elif true == 1 and pred == 1:
            verdaderos_positivos.append(pair)

print("\nFalsos negativos:")
for fn_pair in falsos_negativos:
    print(f"Par: {fn_pair}")

print("\nFalsos positivos:")
for fp_pair in falsos_positivos:
    print(f"Par: {fp_pair}")

print("\nVerdaderos positivos:")
for vp_pair in verdaderos_positivos:
    print(f"Par: {vp_pair}")
       

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5174 - loss: 1.0702 
Pérdida en los nuevos datos: 1.1194
Accuracy en los nuevos datos: 0.4898
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Verdaderos Positivos (TP): 99
Falsos Positivos (FP): 60
Verdaderos Negativos (TN): 36
Falsos Negativos (FN): 1
Precisión (Precision): 0.6226
Exhaustividad (Recall): 0.9900
F1-Score: 0.7645

Falsos negativos:
Par: ['HULK' 'n-omino-main']

Falsos positivos:
Par: ['Second_Project_Dominoes-main' 'copy_of_Moist-Cat_moogle-2023']
Par: ['Moist-Cat_moogle-2023' 'copy_of_Moist-Cat_moogle-2023']
Par: ['Moist-Cat_moogle-2023' 'copy_of_Dominoes-main']
Par: ['project_HULK' 'copy_of_project_HULK']
Par: ['copy_of_LoLProM_Moogle' 'copy_of_Proyecto-II-H.U.L.K-main']
Par: ['copy_of_Moogle_Abraham-master' 'FinalGeoWallE']
Par: ['Dominoes-main' 'copy_of_Dominoes-main']
Par: ['copy_of_MaureenSales_moogle' 'Proyecto-Programacion-Mad-Kev-main']
Par: ['n-omino-main' 'copy_of_Srdariansitop_moogle']
Par: ['copy_of_Dominoes-main' 'c

In [11]:
metrics = siamese_net.evaluate(X1_test, X2_test, y_test, names)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5174 - loss: 1.0702 
Pérdida en los nuevos datos: 1.1194
Accuracy en los nuevos datos: 0.4898
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
Verdaderos Positivos (TP): 99
Falsos Positivos (FP): 60
Verdaderos Negativos (TN): 36
Falsos Negativos (FN): 1
Precisión (Precision): 0.6226
Exhaustividad (Recall): 0.9900
F1-Score: 0.7645

Falsos negativos:
Par: ['HULK' 'n-omino-main']

Falsos positivos:
Par: ['Second_Project_Dominoes-main' 'copy_of_Moist-Cat_moogle-2023']
Par: ['Moist-Cat_moogle-2023' 'copy_of_Moist-Cat_moogle-2023']
Par: ['Moist-Cat_moogle-2023' 'copy_of_Dominoes-main']
Par: ['project_HULK' 'copy_of_project_HULK']
Par: ['copy_of_LoLProM_Moogle' 'copy_of_Proyecto-II-H.U.L.K-main']
Par: ['copy_of_Moogle_Abraham-master' 'FinalGeoWallE']
Par: ['Dominoes-main' 'copy_of_Dominoes-main']
Par: ['copy_of_MaureenSales_moogle' 'Proyecto-Programacion-Mad-Kev-main']
Par: ['n-omino-main' 'copy_of_Srdariansitop_moogle']
Par: ['copy_of_Dominoes-main' 'c

## Hacer una predicción

In [8]:
code1_features = np.random.rand(65)  # Ejemplo de vector de características
code2_features = np.random.rand(65)  # Ejemplo de vector de características

similarity = siamese_net.predict_similarity(code1_features, code2_features)

# Interpretar el resultado
if similarity < 0.5:
    print(f"Los códigos son similares con una probabilidad de {1-similarity:.2f}")
else:
    print(f"Los códigos son diferentes con una probabilidad de {similarity:.2f}")

print(f"La similitud entre los dos códigos es: {similarity}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Los códigos son similares con una probabilidad de 0.58
La similitud entre los dos códigos es: 0.4201033115386963


## Buscando la mejor semilla para el modelo

In [9]:
import os
import tensorflow as tf
import numpy as np
import random

# Definir un rango de semillas
SEEDS = range(50)  # Probar con semillas del 0 al 49

# Variables para almacenar los mejores resultados
best_accuracy = 0
best_precision = 0
best_seed = None
best_model = None

for seed in SEEDS:
    print(f"Probando con semilla: {seed}")
    
    # Configurar la semilla
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    
    # Crear y entrenar el modelo
    snn = SiameseNeuralNetwork(input_shape=(65,), learning_rate=0.001, SEED=seed)
    snn.train(X1_train, X2_train, y_train, validation_data=(X1_val, X2_val, y_val), epochs=10, batch_size=32)
    
    # Evaluar el modelo
    metrics = snn.evaluate(X1_new, X2_new, y_new)
    accuracy = metrics["accuracy"]
    precision = metrics["precision"]
    
    # Comparar con los mejores resultados
    if  ( precision > best_precision):
        best_accuracy = accuracy
        best_precision = precision
        best_seed = seed
        best_model = snn.model  # Guardar el modelo actual como el mejor

# Imprimir los resultados
print(f"Mejor semilla: {best_seed}")
print(f"Mejor accuracy: {best_accuracy:.4f}")
print(f"Mejor precisión: {best_precision:.4f}")

# Guardar el mejor modelo
tf.keras.config.enable_unsafe_deserialization()
best_model.save("models/best_model_all_dataset_val.keras")


Probando con semilla: 0
Epoch 1/10


/home/chony/.local/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.4882 - loss: 2.1467 - val_accuracy: 0.4337 - val_loss: 1.4669
Epoch 2/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4853 - loss: 1.4533 - val_accuracy: 0.4235 - val_loss: 1.3177
Epoch 3/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4959 - loss: 1.3570 - val_accuracy: 0.4949 - val_loss: 1.2727
Epoch 4/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5273 - loss: 1.3169 - val_accuracy: 0.5204 - val_loss: 1.2395
Epoch 5/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5377 - loss: 1.2442 - val_accuracy: 0.5204 - val_loss: 1.2012
Epoch 6/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4961 - loss: 1.2889 - val_accuracy: 0.5051 - val_loss: 1.1623
Epoch 7/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5538 - loss: 1.2474 - val_accuracy: 0.5153 - val_loss: 1.1413
Epoch 8/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5380 - loss: 1.2053 - val_accuracy: 0.5204 - val_loss: 1.1315
Ep

/home/chony/.local/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.5089 - loss: 1.7199 - val_accuracy: 0.4439 - val_loss: 1.4134
Epoch 2/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5146 - loss: 1.3809 - val_accuracy: 0.5000 - val_loss: 1.2964
Epoch 3/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5344 - loss: 1.3232 - val_accuracy: 0.5102 - val_loss: 1.2379
Epoch 4/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5396 - loss: 1.3051 - val_accuracy: 0.5306 - val_loss: 1.2051
Epoch 5/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5411 - loss: 1.2309 - val_accuracy: 0.5255 - val_loss: 1.1764
Epoch 6/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5421 - loss: 1.2122 - val_accuracy: 0.5255 - val_loss: 1.1444
Epoch 7/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5407 - loss: 1.2288 - val_accuracy: 0.5255 - val_loss: 1.1396
Epoch 8/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5453 - loss: 1.2007 - val_accuracy: 0.5204 - val_loss: 1.1303
Ep

...

19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.4902 - loss: 1.6019 - val_accuracy: 0.4796 - val_loss: 1.3207
Epoch 2/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5011 - loss: 1.3497 - val_accuracy: 0.5255 - val_loss: 1.2571
Epoch 3/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5390 - loss: 1.3148 - val_accuracy: 0.5102 - val_loss: 1.2053
Epoch 4/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5314 - loss: 1.2808 - val_accuracy: 0.5102 - val_loss: 1.1898
Epoch 5/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5300 - loss: 1.2408 - val_accuracy: 0.5051 - val_loss: 1.1598
Epoch 6/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5470 - loss: 1.1914 - val_accuracy: 0.5204 - val_loss: 1.1321
Epoch 7/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5333 - loss: 1.2064 - val_accuracy: 0.5306 - val_loss: 1.1170
Epoch 8/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5450 - loss: 1.2084 - val_accuracy: 0.5255 - val_loss: 1.1145
Ep

/home/chony/.local/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.4823 - loss: 1.4055 - val_accuracy: 0.5306 - val_loss: 1.3150
Epoch 2/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5218 - loss: 1.3449 - val_accuracy: 0.5255 - val_loss: 1.2571
Epoch 3/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5288 - loss: 1.2787 - val_accuracy: 0.5153 - val_loss: 1.2229
Epoch 4/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5306 - loss: 1.2397 - val_accuracy: 0.5102 - val_loss: 1.1869
Epoch 5/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5448 - loss: 1.2084 - val_accuracy: 0.5153 - val_loss: 1.1760
Epoch 6/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5483 - loss: 1.1842 - val_accuracy: 0.5102 - val_loss: 1.1528
Epoch 7/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5335 - loss: 1.1627 - val_accuracy: 0.5204 - val_loss: 1.1326
Epoch 8/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5423 - loss: 1.1800 - val_accuracy: 0.5204 - val_loss: 1.1301
Ep

/home/chony/.local/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.4691 - loss: 1.6464 - val_accuracy: 0.4898 - val_loss: 1.3020
Epoch 2/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4977 - loss: 1.3716 - val_accuracy: 0.5102 - val_loss: 1.2593
Epoch 3/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5530 - loss: 1.2735 - val_accuracy: 0.5051 - val_loss: 1.2243
Epoch 4/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5609 - loss: 1.2426 - val_accuracy: 0.5153 - val_loss: 1.2000
Epoch 5/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5481 - loss: 1.2152 - val_accuracy: 0.5102 - val_loss: 1.1713
Epoch 6/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5271 - loss: 1.2093 - val_accuracy: 0.5102 - val_loss: 1.1389
Epoch 7/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5394 - loss: 1.1944 - val_accuracy: 0.5153 - val_loss: 1.1546
Epoch 8/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5778 - loss: 1.1921 - val_accuracy: 0.5102 - val_loss: 1.1237
Ep